In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from bs4 import BeautifulSoup
import requests 

# Compiling mutual fund data from all pages of Groww Investment platform

In [6]:
final=pd.DataFrame(columns=['Mutual Fund Name','fund_rating','return_1yr',
      'return_3yr','return_5yr',
      'risk_type','category'])

final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Mutual Fund Name  0 non-null      object
 1   fund_rating       0 non-null      object
 2   return_1yr        0 non-null      object
 3   return_3yr        0 non-null      object
 4   return_5yr        0 non-null      object
 5   risk_type         0 non-null      object
 6   category          0 non-null      object
dtypes: object(7)
memory usage: 0.0+ bytes


In [9]:
for j in range(73):
    url='https://groww.in/mutual-funds/filter?q=&fundSize=&pageNo={}&sortBy=0'.format(j)
    page=requests.get(url)
    soup=BeautifulSoup(page.content,'html.parser')
 
  #  fund containers
    fund=soup.find_all('tr',class_="f22Card")

  #scraping all field from each section
    name=[]
    rating=[]
    return_1yr=[]
    return_3yr=[]
    return_5yr=[]
    risk_type=[]
    category=[]
    

    for i in fund:
        name.append(i.find('div',class_="fs14 clrText fw500 f22LH34 f22Mb4 truncate").text.strip())
        return_1yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[0].text.strip())
        return_3yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[1].text.strip())
        return_5yr.append(i.find_all('div',class_="fs14 clrText fw500 center-align f22Mb4")[2].text.strip())
        risk_type.append(i.find('div',class_="fs12 fw500 clrSubText f22Ls2").text.strip())
        category.append(i.find('div',class_="fs12 fw500clrSubText f22Ls2").text.strip())
        try: 
              rating.append(i.find('div',class_="fs12 fw500 clrSubText f22Ls2 valign-wrapper").text.strip())
        except:
              rating.append(np.nan)

    d={'Mutual Fund Name':name,'fund_rating':rating,'return_1yr':return_1yr,
      'return_3yr':return_3yr,'return_5yr':return_5yr,
      'risk_type':risk_type,'category':category} 
    
    df=pd.DataFrame(d)

    #combining dataframes from each page
    final = pd.concat([final,df],ignore_index=True)  

In [12]:
final.to_parquet('mutual_funds.gzip', compression='gzip')

# Get Historical Data

In [28]:
daterange = pd.date_range(start="2022-05-01", end="2023-04-30", freq='B').strftime('%d-%b-%Y').tolist() 


In [29]:
from IPython.display import clear_output
from io import StringIO
final = pd.DataFrame(columns=['Scheme Code','Scheme Name','ISIN Div Payout/ISIN Growth','ISIN Div Reinvestment','Net Asset Value','Repurchase Price','Sale Price','Date'])

for date in daterange:
    clear_output(wait=True)
    hurl = f'https://portal.amfiindia.com/DownloadNAVHistoryReport_Po.aspx?tp=1&frmdt={date}'
    try:
        data = requests.get(hurl)
    except:
        continue
    file1 = []
    for line in data.text.split('\n'):
        if line.find(';')!=-1: file1.append(line)
        else: continue 
    ssvfile = '\n'.join(file1)
    df = pd.read_csv(StringIO(ssvfile), sep=";")
    final = pd.concat([final, df], ignore_index=True)
    print(date)
    

28-Apr-2023


In [30]:
final['Date'] = pd.to_datetime(final['Date'])
final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563130 entries, 0 to 1563129
Data columns (total 8 columns):
 #   Column                       Non-Null Count    Dtype         
---  ------                       --------------    -----         
 0   Scheme Code                  1563130 non-null  object        
 1   Scheme Name                  1563130 non-null  object        
 2   ISIN Div Payout/ISIN Growth  1402799 non-null  object        
 3   ISIN Div Reinvestment        804131 non-null   object        
 4   Net Asset Value              1563130 non-null  object        
 5   Repurchase Price             0 non-null        float64       
 6   Sale Price                   0 non-null        float64       
 7   Date                         1563130 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 95.4+ MB


In [31]:
final['Year'] = final['Date'].dt.year
final['Month'] = final['Date'].dt.month
final['Day'] = final['Date'].dt.day

In [32]:
#final.info()
#final.sample(5)
final = final.drop(columns=['ISIN Div Payout/ISIN Growth','ISIN Div Reinvestment','Repurchase Price','Sale Price','Date'])

In [33]:
final['Net Asset Value'] = final['Net Asset Value'].replace('N.A.', np.nan).dropna()

final['Net Asset Value'] = pd.to_numeric(final['Net Asset Value'])
final.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1563130 entries, 0 to 1563129
Data columns (total 6 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Scheme Code      1563130 non-null  object 
 1   Scheme Name      1563130 non-null  object 
 2   Net Asset Value  1563126 non-null  float64
 3   Year             1563130 non-null  int64  
 4   Month            1563130 non-null  int64  
 5   Day              1563130 non-null  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 71.6+ MB


In [34]:
final.to_parquet('mutual_funds_historical2023.gzip', compression='gzip', partition_cols=['Year', 'Month', 'Day'])

# Combining Parquets

In [35]:
df2021 = pd.read_parquet('mutual_funds_historical2021.gzip')
df2022 = pd.read_parquet('mutual_funds_historical2022.gzip')
df2023 = pd.read_parquet('mutual_funds_historical2023.gzip')

finalhs = pd.concat([df2021,df2022,df2023], ignore_index=True)

In [37]:
finalhs.to_parquet('mutual_funds_historical.gzip', compression='gzip', partition_cols=['Year', 'Month', 'Day'])

# Starting analysis

In [5]:
df = pd.read_parquet("mutual_funds_historical.gzip", columns=["Scheme Name", "Net Asset Value", "Day", "Month", "Year"])
df.sample(10)

,Scheme Name,Net Asset Value,Day,Month,Year
1645851,Aditya Birla Sun Life International Equity Fun...,28.7659,10,3,2021
587894,NIPPON INDIA EQUITY SAVINGS FUND - MONTHLY IDC...,8.2177,3,6,2020
4006327,HSBC CRISIL IBX 50:50 Gilt Plus SDL Apr 2028 I...,10.0145,23,9,2022
1837194,Axis Special Situations Fund - Regular Plan - ...,10.8200,29,4,2021
207449,Nippon India Growth Fund-Growth Plan-Bonus Option,195.6211,3,11,2020
4337891,ITI Banking & PSU Debt Fund - Direct Plan - ID...,11.1706,1,3,2023
4323317,Baroda BNP Paribas Ultra Short Duratio Fund- R...,1000.1132,8,2,2023
585159,HSBC Low Duration Fund - Regular - Growth,20.7846,3,6,2020
4522426,ICICI Prudential Nifty 100 Low Volatility 30 E...,11.6955,20,4,2023
4172941,WhiteOak Capital Flexi Cap Fund Direct Plan Gr...,10.1500,30,1,2023


In [8]:
cols=["Year", "Month", "Day"]
df["Date"] = df[cols].apply(lambda x: "-".join(x.values.astype(str)), axis="columns")
df["Date"] = pd.to_datetime(df["Date"])

In [12]:
df= df.drop(["Day", "Month", "Year"], axis=1)

In [15]:
df["Date"] = df["Date"].apply(lambda x: pd.Timestamp.timestamp(x))

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4582978 entries, 0 to 4582977
Data columns (total 3 columns):
 #   Column           Dtype  
---  ------           -----  
 0   Scheme Name      object 
 1   Net Asset Value  float64
 2   Date             float64
dtypes: float64(2), object(1)
memory usage: 104.9+ MB


In [17]:
df.sample(5)

,Scheme Name,Net Asset Value,Date
2544726,BNP PARIBAS LOW DURATION Fund-Regular Plan-Gro...,32.5996,1.642032e+09
3005104,Franklin India Equity Hybrid Fund - Direct - I...,29.2031,1.672358e+09
1828423,Invesco India Gilt Fund - Direct Plan - Monthl...,1879.3836,1.619482e+09
4046302,SBI Long Term Equity Fund - Direct Plan - Inco...,67.5400,1.662422e+09
562810,PGIM India Credit Risk Fund - Direct Plan - Gr...,14.5513,1.592957e+09
